# Fashion CNN 정의

# 중요 수식
### W2 = ((W1 - k_size + 2P) / S ) + 1 
### H2 = ((H1 - k_size + 2P) / S ) + 1 

# 1. 라이브러리 임포트

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

import torchvision
from torchvision import transforms
from torchvision.datasets import FashionMNIST

import numpy as np
from sklearn.metrics import accuracy_score

# 2. Dataset 준비

In [4]:
train_set = FashionMNIST("../0_data/", download=True, train=True, transform=transforms.ToTensor())
test_set = FashionMNIST("../0_data/", download=True, train=False, transform=transforms.ToTensor())

train_loader = DataLoader(train_set, batch_size=64, shuffle=True)
test_loader = DataLoader(test_set, batch_size=64, shuffle=True)

# 3. 모델 생성 및 컴파일
- input_size = (1, 28, 28)
- conv1d = ((28-2 + 0)/ 1 ) + 1 --> (27, 27) --> (13, 13)
- conv2d = ((13-2 + 0)/ 1 ) + 1 --> (12, 12) --> (6, 6)

In [18]:
class FashionCNN(nn.Module):
    def __init__(self):
        super(FashionCNN, self).__init__()
        self.conv1d = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=(2,2), stride=1, padding=0),
            nn.BatchNorm2d(32),
            nn.MaxPool2d((2,2)),
            nn.ReLU(inplace=True)
        )
        self.conv2d = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(2,2), stride=1, padding=0),
            nn.BatchNorm2d(64),
            nn.MaxPool2d((2,2)),
            nn.ReLU(inplace=True)
        )
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64*6*6, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, 32),
            nn.ReLU(inplace=True),
            nn.Linear(32,10),
            nn.Softmax(dim=1)
        )
    def forward(self, x):
        x = self.conv1d(x)
        x = self.conv2d(x)
        x = self.fc(x)
        return x

In [19]:
model = FashionCNN()

optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
criterion = nn.CrossEntropyLoss()

In [22]:
def train(model, dataloader, optimizer, criterion):
    model.train()
    total_loss = 0
    for data in (dataloader):
        inputs, targs = data

        optimizer.zero_grad()

        y_pred = model(inputs)
        loss = criterion(y_pred, targs)
        total_loss += loss

        loss.backward()
        optimizer.step()
    
    return total_loss / len(dataloader)


def eval(model, dataloader, criterion):
    model.eval()

    y_pred_lst = []
    y_targ_lst = []
    loss = 0
    for idx, data in (enumerate(dataloader)):
        inputs, targs = data

        y_pred = model(inputs)

        loss += criterion(y_pred, targs)

        class_pred = torch.argmax(y_pred, dim=1)
        
        for i in class_pred:
            y_pred_lst.append(i.to("cpu").item())

        for i in targs:
            y_targ_lst.append(i.to("cpu").item())
    
    return y_targ_lst, y_pred_lst

In [23]:
for epoch in range(10):
    train_loss = train(model, train_loader, optimizer, criterion)
    
    print(f"Epoch : {epoch} \t loss : {train_loss:.6f}")

y_targ, y_pred = eval(model, test_loader, criterion)
print(f"정확도 : {accuracy_score(y_targ, y_pred):.2f}%")

Epoch : 0 	 loss : 1.639245
Epoch : 1 	 loss : 1.590606
Epoch : 2 	 loss : 1.577991
Epoch : 3 	 loss : 1.571548
Epoch : 4 	 loss : 1.568098
Epoch : 5 	 loss : 1.562429
Epoch : 6 	 loss : 1.558503
Epoch : 7 	 loss : 1.557612
Epoch : 8 	 loss : 1.552572
Epoch : 9 	 loss : 1.551498
정확도 : 0.90%
